In [ ]:
import os
import sys
os.environ["PYSPARK_PYTHON"]='/opt/anaconda/envs/bd9/bin/python'
os.environ["SPARK_HOME"]='/usr/hdp/current/spark2-client'
os.environ["PYSPARK_SUBMIT_ARGS"]='--num-executors 5 --executor-memory 4g --executor-cores 1 --driver-memory 2g pyspark-shell'

spark_home = os.environ.get('SPARK_HOME', None)
if not spark_home:
    raise ValueError('SPARK_HOME environment variable is not set')

sys.path.insert(0, os.path.join(spark_home, 'python'))
sys.path.insert(0, os.path.join(spark_home, 'python/lib/py4j-0.10.7-src.zip'))
exec(open(os.path.join(spark_home, 'python/pyspark/shell.py')).read())

In [ ]:
from pyspark import SparkConf
from pyspark.sql import SparkSession

conf = SparkConf()

spark = SparkSession.builder.config(conf=conf).getOrCreate()

In [ ]:
spark

In [4]:
! hdfs dfs -ls /labs/slaba03/

Found 4 items
-rw-r--r--   3 hdfs hdfs   91066524 2022-01-06 18:46 /labs/slaba03/laba03_items.csv
-rw-r--r--   3 hdfs hdfs   29965581 2022-01-06 18:46 /labs/slaba03/laba03_test.csv
-rw-r--r--   3 hdfs hdfs   74949368 2022-01-06 18:46 /labs/slaba03/laba03_train.csv
-rw-r--r--   3 hdfs hdfs  871302535 2022-01-06 18:46 /labs/slaba03/laba03_views_programmes.csv


In [73]:
from pyspark.sql.types import StructType, StructField, StringType, IntegerType, FloatType, ArrayType
from pyspark.sql import functions as F
from pyspark.sql.functions import udf

In [6]:
from pyspark.ml.feature import Tokenizer, StopWordsRemover, CountVectorizer, IDF, HashingTF
from pyspark.ml import Pipeline

# train

In [7]:
schema = StructType([
    StructField("user_id", IntegerType(), True),
    StructField("item_id", IntegerType(), True),
    StructField("purchase", IntegerType(), True),
])    

In [8]:
df_user = spark.read.csv("/labs/slaba03/laba03_train.csv", schema=schema, header=True, multiLine=True, escape='"')

In [9]:
df_user.show()

+-------+-------+--------+
|user_id|item_id|purchase|
+-------+-------+--------+
|   1654|  74107|       0|
|   1654|  89249|       0|
|   1654|  99982|       0|
|   1654|  89901|       0|
|   1654| 100504|       0|
|   1654|  66187|       0|
|   1654|  84350|       0|
|   1654|  92854|       0|
|   1654|  72811|       0|
|   1654|  86876|       0|
|   1654| 102657|       0|
|   1654| 100482|       0|
|   1654|  89677|       0|
|   1654|  99419|       0|
|   1654|  66603|       0|
|   1654|   7363|       0|
|   1654|   1320|       0|
|   1654|  88892|       0|
|   1654|  66671|       0|
|   1654|  75925|       0|
+-------+-------+--------+
only showing top 20 rows



# test

In [10]:
schema = StructType([
    StructField("user_id", IntegerType(), True),
    StructField("item_id", IntegerType(), True),
]) 

In [11]:
df_user_test = spark.read.csv("/labs/slaba03/laba03_test.csv", schema=schema, header=True, multiLine=True, escape='"')

In [12]:
df_user_test.show()

+-------+-------+
|user_id|item_id|
+-------+-------+
|   1654|  94814|
|   1654|  93629|
|   1654|   9980|
|   1654|  95099|
|   1654|  11265|
|   1654|  88896|
|   1654|  67740|
|   1654|  74271|
|   1654|  99871|
|   1654|  78570|
|   1654|  71942|
|   1654|  74367|
|   1654|  98628|
|   1654|  95887|
|   1654|  77795|
|   1654|  75152|
|   1654|  74905|
|   1654|   9068|
|   1654|  72954|
|   1654| 102431|
+-------+-------+
only showing top 20 rows



# items

In [13]:
items_schema = StructType([
    StructField("item_id", IntegerType()),
    StructField("channel_id", IntegerType()),
    StructField("datetime_availability_start", StringType()),
    StructField("datetime_availability_stop", StringType()),
    StructField("datetime_show_start", StringType()),
    StructField("datetime_show_stop", StringType()),
    StructField("content_type", IntegerType()),
    StructField("title", StringType(), nullable=True),
    StructField("year", FloatType(), nullable=True),
    StructField("genres", StringType()),
    StructField("region_id", IntegerType()),
])

In [14]:
df_items = spark.read.option('delimiter', '\t').csv("/labs/slaba03/laba03_items.csv", header=True, schema=items_schema, multiLine=True, escape='"')

In [15]:
df_items.show()

+-------+----------+---------------------------+--------------------------+-------------------+------------------+------------+--------------------+------+--------------------+---------+
|item_id|channel_id|datetime_availability_start|datetime_availability_stop|datetime_show_start|datetime_show_stop|content_type|               title|  year|              genres|region_id|
+-------+----------+---------------------------+--------------------------+-------------------+------------------+------------+--------------------+------+--------------------+---------+
|  65667|      null|       1970-01-01T00:00:00Z|      2018-01-01T00:00:00Z|               null|              null|           1|на пробах только ...|2013.0|             Эротика|     null|
|  65669|      null|       1970-01-01T00:00:00Z|      2018-01-01T00:00:00Z|               null|              null|           1|скуби ду: эротиче...|2011.0|             Эротика|     null|
|  65668|      null|       1970-01-01T00:00:00Z|      2018-01-01T

In [16]:
df_items = df_items.filter(F.col("content_type") == 1)
df_items = df_items.select(F.regexp_replace('title', r',|\.|&|\\|\||–|_|:|-|\!|\?', '').alias('title'), 'item_id', 'year', 'genres')


In [17]:
from pyspark.sql.functions import split, when, array
df_items2 = df_items.withColumn("genres_word", split("genres", ","))
df_items2  = df_items2.withColumn('genres_word', when(df_items2['genres_word'].isNull(), array().cast("array<integer>")).otherwise(df_items2['genres_word']))

In [18]:
df_items2.show()

+--------------------+-------+------+--------------------+--------------------+
|               title|item_id|  year|              genres|         genres_word|
+--------------------+-------+------+--------------------+--------------------+
|на пробах только ...|  65667|2013.0|             Эротика|           [Эротика]|
|скуби ду эротичес...|  65669|2011.0|             Эротика|           [Эротика]|
|горячие девочки д...|  65668|2011.0|             Эротика|           [Эротика]|
|соблазнительницы ...|  65671|2011.0|             Эротика|           [Эротика]|
|секретные сексмат...|  65670|2010.0|             Эротика|           [Эротика]|
|      все о мужчинах|  65809|2016.0|             Комедии|           [Комедии]|
|8 лучших свиданий...|  65810|2016.0|   Комедии,Мелодрамы|[Комедии, Мелодрамы]|
|            византия|    326|2012.0|Ужасы,Триллеры,Др...|[Ужасы, Триллеры,...|
|девственники бере...|    336|2012.0|Ужасы,Комедии,Фан...|[Ужасы, Комедии, ...|
|   8 первых свиданий|    357|2012.0|Ком

In [19]:
stop_words = StopWordsRemover.loadDefaultStopWords("russian")  
tokenizer = Tokenizer(inputCol="title", outputCol="title_words")
swr = StopWordsRemover(inputCol=tokenizer.getOutputCol(), outputCol="title_words_filtered", stopWords=stop_words)
#count_vectorizer = CountVectorizer(inputCol=swr.getOutputCol(), outputCol="word_vector", vocabSize=10000)
#tfidf = IDF(inputCol="word_vector", outputCol="tfidf")    

hasher = HashingTF(numFeatures=200, binary=True, inputCol=swr.getOutputCol(), outputCol="word_vector")


preprocessing = Pipeline(stages=[
        tokenizer,
        swr,
        #count_vectorizer,
        #tfidf
        hasher
    ])
    
preprocessing_model = preprocessing.fit(df_items2)
df_items3 = preprocessing_model.transform(df_items2)

In [20]:
df_items3.show(2, vertical=True)

-RECORD 0------------------------------------
 title                | на пробах только ... 
 item_id              | 65667                
 year                 | 2013.0               
 genres               | Эротика              
 genres_word          | [Эротика]            
 title_words          | [на, пробах, толь... 
 title_words_filtered | [пробах, девушки,... 
 word_vector          | (200,[0,4,23,29,8... 
-RECORD 1------------------------------------
 title                | скуби ду эротичес... 
 item_id              | 65669                
 year                 | 2011.0               
 genres               | Эротика              
 genres_word          | [Эротика]            
 title_words          | [скуби, ду, эроти... 
 title_words_filtered | [скуби, ду, эроти... 
 word_vector          | (200,[17,33,99,10... 
only showing top 2 rows



In [21]:
count_vectorizer = CountVectorizer(inputCol='genres_word', outputCol='genre_vector')
items = count_vectorizer.fit(df_items3).transform(df_items3)

In [22]:
items.show()

+--------------------+-------+------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+
|               title|item_id|  year|              genres|         genres_word|         title_words|title_words_filtered|         word_vector|        genre_vector|
+--------------------+-------+------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+
|на пробах только ...|  65667|2013.0|             Эротика|           [Эротика]|[на, пробах, толь...|[пробах, девушки,...|(200,[0,4,23,29,8...|     (83,[20],[1.0])|
|скуби ду эротичес...|  65669|2011.0|             Эротика|           [Эротика]|[скуби, ду, эроти...|[скуби, ду, эроти...|(200,[17,33,99,10...|     (83,[20],[1.0])|
|горячие девочки д...|  65668|2011.0|             Эротика|           [Эротика]|[горячие, девочки...|[горячие, девочки...|(200,[31,51,67,93...|     (83,[20],[1.0])|
|соблазнительниц

# обьединяем train и items

In [23]:
df_train = df_user.join(items, on="item_id", how="inner")

In [24]:
df_train.show(2,vertical=True)

-RECORD 0------------------------------------
 item_id              | 8389                 
 user_id              | 520446               
 purchase             | 0                    
 title                | пес в сапогах (су... 
 year                 | 1981.0               
 genres               | Мультфильмы,Детск... 
 genres_word          | [Мультфильмы, Дет... 
 title_words          | [пес, в, сапогах,... 
 title_words_filtered | [пес, сапогах, (с... 
 word_vector          | (200,[58,84,187],... 
 genre_vector         | (83,[6,14,19,23],... 
-RECORD 1------------------------------------
 item_id              | 8389                 
 user_id              | 556825               
 purchase             | 0                    
 title                | пес в сапогах (су... 
 year                 | 1981.0               
 genres               | Мультфильмы,Детск... 
 genres_word          | [Мультфильмы, Дет... 
 title_words          | [пес, в, сапогах,... 
 title_words_filtered | [пес, сапо

In [25]:
test = df_user_test.join(items, on="item_id", how="inner")

In [26]:
test.show(2,vertical=True)

-RECORD 0------------------------------------
 item_id              | 8389                 
 user_id              | 1654                 
 title                | пес в сапогах (су... 
 year                 | 1981.0               
 genres               | Мультфильмы,Детск... 
 genres_word          | [Мультфильмы, Дет... 
 title_words          | [пес, в, сапогах,... 
 title_words_filtered | [пес, сапогах, (с... 
 word_vector          | (200,[58,84,187],... 
 genre_vector         | (83,[6,14,19,23],... 
-RECORD 1------------------------------------
 item_id              | 8389                 
 user_id              | 510087               
 title                | пес в сапогах (су... 
 year                 | 1981.0               
 genres               | Мультфильмы,Детск... 
 genres_word          | [Мультфильмы, Дет... 
 title_words          | [пес, в, сапогах,... 
 title_words_filtered | [пес, сапогах, (с... 
 word_vector          | (200,[58,84,187],... 
 genre_vector         | (83,[6,14,

## фичи

In [ ]:
#df_val = df_train.where(df_train.year<2020)

In [27]:
df_val2 = df_train.groupby(F.col('item_id')).agg(F.mean('purchase').alias("item_mean_encoding"))

In [28]:
df_val3 = df_train.groupby(F.col('user_id')).agg(F.mean('purchase').alias("intensity_user"))

In [29]:
df_val3.show()

+-------+--------------------+
|user_id|      intensity_user|
+-------+--------------------+
| 754230|0.027575641516660282|
| 761341|3.875968992248062E-4|
| 776188|0.001152516327314637|
| 780033|7.757951900698216E-4|
| 798454|3.840245775729646...|
| 825061|0.001931247585940...|
| 833685|0.007500986971969996|
| 846231|0.001923816852635629|
| 851486|                 0.0|
| 867850|3.829950210647261...|
| 870928|7.674597083653108E-4|
| 879401|0.004283489096573208|
| 901457|                 0.0|
| 927211|3.916960438699569E-4|
| 928140|3.869969040247678E-4|
| 793876|0.001940240589833...|
| 795620|0.004243827160493827|
| 822709|3.789314134141720...|
| 824008|3.821169277799006...|
| 851848|3.888024883359253...|
+-------+--------------------+
only showing top 20 rows



In [30]:
df_train2 = df_train.join(df_val2, on="item_id", how="left")

In [31]:
df_train3 = df_train2.join(df_val3, on="user_id", how="left")

In [32]:
df_train3.show()

+-------+-------+--------+--------------------+------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+
|user_id|item_id|purchase|               title|  year|              genres|         genres_word|         title_words|title_words_filtered|         word_vector|        genre_vector|  item_mean_encoding|      intensity_user|
+-------+-------+--------+--------------------+------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+
| 754230|   8389|       0|пес в сапогах (су...|1981.0|Мультфильмы,Детск...|[Мультфильмы, Дет...|[пес, в, сапогах,...|[пес, сапогах, (с...|(200,[58,84,187],...|(83,[6,14,19,23],...|0.005979073243647235|0.027575641516660282|
| 754230|   8638|       1| охотники на демонов|2012.0|Ужасы,Комедии,Три...|[Ужасы, Комедии, ...|[охотники, н

In [33]:
test = test.join(df_val2, on="item_id", how="left")

In [34]:
test = test.join(df_val3, on="user_id", how="left")

In [35]:
test.show(2,vertical=True)

-RECORD 0------------------------------------
 user_id              | 754230               
 item_id              | 73041                
 title                | коко до шанель       
 year                 | 2009.0               
 genres               | Драмы,Исторически... 
 genres_word          | [Драмы, Историчес... 
 title_words          | [коко, до, шанель]   
 title_words_filtered | [коко, шанель]       
 word_vector          | (200,[63,108],[1.... 
 genre_vector         | (83,[0,1,25],[1.0... 
 item_mean_encoding   | 7.352941176470588E-4 
 intensity_user       | 0.027575641516660282 
-RECORD 1------------------------------------
 user_id              | 754230               
 item_id              | 74440                
 title                | области тьмы         
 year                 | 2011.0               
 genres               | Детективы,Фантаст... 
 genres_word          | [Детективы, Фанта... 
 title_words          | [области, тьмы]      
 title_words_filtered | [области, 

In [36]:
df_train3 = df_train3.na.fill(0)

In [37]:
test = test.na.fill(0)

# обьединение фичей

In [38]:
from pyspark.ml.feature import VectorAssembler

In [39]:
assembler = VectorAssembler(inputCols=['user_id','item_id','item_mean_encoding', 'intensity_user'], outputCol="features")

In [40]:
df_train = assembler.transform(df_train3)

In [41]:
df_train.show()

+-------+-------+--------+--------------------+------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+
|user_id|item_id|purchase|               title|  year|              genres|         genres_word|         title_words|title_words_filtered|         word_vector|        genre_vector|  item_mean_encoding|      intensity_user|            features|
+-------+-------+--------+--------------------+------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+
| 754230|   8389|       0|пес в сапогах (су...|1981.0|Мультфильмы,Детск...|[Мультфильмы, Дет...|[пес, в, сапогах,...|[пес, сапогах, (с...|(200,[58,84,187],...|(83,[6,14,19,23],...|0.005979073243647235|0.027575641516660282|[754230.0,8389.0,...|
| 754230|   8638|       

In [42]:
test = assembler.transform(test)

In [43]:
test.show(vertical=True)

-RECORD 0------------------------------------
 user_id              | 754230               
 item_id              | 93486                
 title                | виноградник          
 year                 | 2014.0               
 genres               | Триллеры,Фантастика  
 genres_word          | [Триллеры, Фантас... 
 title_words          | [виноградник]        
 title_words_filtered | [виноградник]        
 word_vector          | (200,[33],[1.0])     
 genre_vector         | (83,[3,12],[1.0,1... 
 item_mean_encoding   | 0.002141327623126... 
 intensity_user       | 0.027575641516660282 
 features             | [754230.0,93486.0... 
-RECORD 1------------------------------------
 user_id              | 754230               
 item_id              | 94819                
 title                | невидимый мир        
 year                 | 2011.0               
 genres               | Драмы,Зарубежные     
 genres_word          | [Драмы, Зарубежные]  
 title_words          | [невидимый

In [44]:
from pyspark.ml.feature import MinMaxScaler
scaler = MinMaxScaler(inputCol='features', outputCol='features_minmax')
scaler_model = scaler.fit(df_train)
items1 = scaler_model.transform(df_train)

In [45]:
items1.show(vertical=True)

-RECORD 0------------------------------------
 user_id              | 754230               
 item_id              | 540                  
 purchase             | 0                    
 title                | мой парень  псих     
 year                 | 2012.0               
 genres               | Комедии,Драмы,Мел... 
 genres_word          | [Комедии, Драмы, ... 
 title_words          | [мой, парень, , п... 
 title_words_filtered | [парень, , псих]     
 word_vector          | (200,[44,164,172]... 
 genre_vector         | (83,[0,1,2,7],[1.... 
 item_mean_encoding   | 7.23589001447178E-4  
 intensity_user       | 0.027575641516660282 
 features             | [754230.0,540.0,7... 
 features_minmax      | [0.80078655367760... 
-RECORD 1------------------------------------
 user_id              | 754230               
 item_id              | 1522                 
 purchase             | 0                    
 title                | патруль времени      
 year                 | 1994.0    

In [49]:
from pyspark.ml.feature import MinMaxScaler
scaler = MinMaxScaler(inputCol='features', outputCol='features_minmax')
scaler_model = scaler.fit(test)
test = scaler_model.transform(test)

# разбиваем на train и val

In [ ]:
train = df_train.sampleBy("purchase", fractions={0: 0.8, 1: 0.8}, seed=5757)

In [ ]:
val = df_train.join(train, (df_train.item_id == train.item_id) & (df_train.user_id == train.user_id), how="leftanti")

In [ ]:
from pyspark.ml.classification import LogisticRegression

In [ ]:
lr = LogisticRegression(featuresCol='features', labelCol="purchase", maxIter=15)

In [ ]:
lr_model = lr.fit(train)

In [ ]:
predictions = lr_model.transform(val)

In [ ]:
df_train.groupby("purchase").count().collect()

# Обучение

In [65]:
from pyspark.ml.classification import LogisticRegression

In [66]:
lr = LogisticRegression(featuresCol='features_minmax', labelCol="purchase", maxIter=15)

In [67]:
lr_model_final = lr.fit(items1)

In [68]:
predictions = lr_model_final.transform(test)

In [69]:
predictions.show(vertical=True)

-RECORD 0------------------------------------
 user_id              | 754230               
 item_id              | 93486                
 title                | виноградник          
 year                 | 2014.0               
 genres               | Триллеры,Фантастика  
 genres_word          | [Триллеры, Фантас... 
 title_words          | [виноградник]        
 title_words_filtered | [виноградник]        
 word_vector          | (200,[33],[1.0])     
 genre_vector         | (83,[3,12],[1.0,1... 
 item_mean_encoding   | 0.002141327623126... 
 intensity_user       | 0.027575641516660282 
 features             | [754230.0,93486.0... 
 features_minmax      | [0.80078655367760... 
 rawPrediction        | [5.68888967833494... 
 probability          | [0.99662806095286... 
 prediction           | 0.0                  
-RECORD 1------------------------------------
 user_id              | 754230               
 item_id              | 94819                
 title                | невидимый 

In [70]:
prediction = predictions.select('user_id', 'item_id', 'prediction')

In [71]:
prediction.groupby("prediction").count().collect()

[Row(prediction=0.0, count=2156790), Row(prediction=1.0, count=50)]

In [74]:
def proba1(vector):
    from pyspark.ml.linalg import Vectors
   # res = float(vector.dot(Vectors.dense([0.0, 1.0])))
    res = float(vector[1])
    return res
proba1 = udf(proba1, FloatType())

In [75]:
predictions1 = predictions.select('*', proba1('probability').alias('purchase'))

In [77]:
predictions1.show(vertical=True)

-RECORD 0------------------------------------
 user_id              | 754230               
 item_id              | 73041                
 title                | коко до шанель       
 year                 | 2009.0               
 genres               | Драмы,Исторически... 
 genres_word          | [Драмы, Историчес... 
 title_words          | [коко, до, шанель]   
 title_words_filtered | [коко, шанель]       
 word_vector          | (200,[63,108],[1.... 
 genre_vector         | (83,[0,1,25],[1.0... 
 item_mean_encoding   | 7.352941176470588E-4 
 intensity_user       | 0.027575641516660282 
 features             | [754230.0,73041.0... 
 features_minmax      | [0.80078655367760... 
 rawPrediction        | [5.74976825348703... 
 probability          | [0.99682658414767... 
 prediction           | 0.0                  
 purchase             | 0.0031734158         
-RECORD 1------------------------------------
 user_id              | 754230               
 item_id              | 74440     

In [79]:
prediction = predictions1.select('user_id', 'item_id', 'purchase')

In [ ]:
prediction = prediction.withColumnRenamed('prediction', 'purchase')

In [ ]:
prediction.show(vertical=True)

In [80]:
prediction.orderBy(["user_id", "item_id"], ascending=[1, 1]).toPandas().to_csv('lab03.csv')

In [61]:
from pyspark.ml.classification import GBTClassifier
gbt = GBTClassifier(featuresCol='features', labelCol="purchase")

In [62]:
pipeline_model = gbt.fit(items1)

In [63]:
predictions = pipeline_model.transform(test)

In [64]:
predictions.groupby("prediction").count().collect()

[Row(prediction=0.0, count=2156840)]

In [ ]:
from pyspark.ml.evaluation import BinaryClassificationEvaluator
evaluator = BinaryClassificationEvaluator(rawPredictionCol="probability", labelCol="purchase", metricName='areaUnderROC')

In [ ]:
evaluator.evaluate(predictions)

In [ ]:
sc.stop()